<a href="https://colab.research.google.com/github/Natali-bali/Data_Eng_Bootcamp_2024/blob/main/Homework_data_talks_club_data_extraction_and_ingestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [37]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)
sum = 0
for sqrt_value in generator:
    sum = sum + sqrt_value
print(sum)

limit = 11
generator = square_root_generator(limit)
for sqrt_value in generator:
    print( sqrt_value)

8.382332347441762
1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [38]:
import duckdb
import dlt

def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}
#for person in people_1():
#    print(person)
#for person in people_2():
#    print(person)
generator_pipeline = dlt.pipeline(destination = 'duckdb', dataset_name = 'homework')
info = generator_pipeline.run(people_1(),
                              table_name = "persons",
                              write_disposition = "replace")
print(info)
print('\n Pipeline and data set')
print(generator_pipeline.pipeline_name)
print(generator_pipeline.dataset_name)

conn = duckdb.connect(database = f"{generator_pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{generator_pipeline.dataset_name}'")
print('\Created tables in database:')
display(conn.sql("show tables"))

sum = conn.sql("SELECT sum(Age) FROM persons")
print(f"Summ of ages from first generator: \n {sum}")

generator_pipeline.run(people_2(),
                       table_name = "persons",
                       write_disposition = "append")

sum = conn.sql("SELECT sum(Age) FROM persons")
print(f"Summ of ages from first and second generator: \n {sum}")

Pipeline dlt_colab_kernel_launcher load step completed in 0.23 seconds
1 load package(s) were loaded to destination duckdb and into dataset homework
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708259198.0887191 is LOADED and contains no failed jobs

 Pipeline and data set
dlt_colab_kernel_launcher
homework
\Created tables in database:


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ hashed_table        │
│ persons             │
└─────────────────────┘

Summ of ages from first generator: 
 ┌──────────┐
│ sum(Age) │
│  int128  │
├──────────┤
│      140 │
└──────────┘

Summ of ages from first and second generator: 
 ┌──────────┐
│ sum(Age) │
│  int128  │
├──────────┤
│      353 │
└──────────┘



# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


In [43]:
generator_pipeline_key = dlt.pipeline(destination = 'duckdb', dataset_name = 'homework')
info = generator_pipeline_key.run(people_1(),
                              table_name = "hashed_table",
                              write_disposition = "replace",
                              primary_key = 'ID')
print(info)
print('\n Pipeline and data set')
print(generator_pipeline.pipeline_name)
print(generator_pipeline.dataset_name)

conn = duckdb.connect(database = f"{generator_pipeline_key.pipeline_name}.duckdb")

conn.sql(f"SET search_path = '{generator_pipeline_key.dataset_name}'")
print('\Created tables in database:')
display(conn.sql("show tables"))

sum = conn.sql("SELECT sum(Age) FROM hashed_table")
print(f"Summ of ages from first generator: \n {sum}")

generator_pipeline_key.run(people_2(),
                       table_name = "hashed_table",
                       write_disposition = "append",
                       primary_key = "ID")

sum = conn.sql("SELECT sum(Age) FROM hashed_table")
print(f"Summ of ages from first and second generator: \n {sum}")
pers = conn.sql("SELECT * FROM hashed_table").df()
display(pers)

Pipeline dlt_colab_kernel_launcher load step completed in 0.18 seconds
1 load package(s) were loaded to destination duckdb and into dataset homework
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708259329.468253 is LOADED and contains no failed jobs

 Pipeline and data set
dlt_colab_kernel_launcher
homework
\Created tables in database:


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ hashed_table        │
│ persons             │
└─────────────────────┘

Summ of ages from first generator: 
 ┌──────────┐
│ sum(Age) │
│  int128  │
├──────────┤
│      140 │
└──────────┘

Summ of ages from first and second generator: 
 ┌──────────┐
│ sum(Age) │
│  int128  │
├──────────┤
│      353 │
└──────────┘



,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708259329.468253,xC06uBVsGcNDgA,None
1,2,Person_2,27,City_A,1708259329.468253,N85x20srlnTwFw,None
2,3,Person_3,28,City_A,1708259329.468253,3fOHh8m10ZqaWQ,None
3,4,Person_4,29,City_A,1708259329.468253,R7z2MWgNufM9+g,None
4,5,Person_5,30,City_A,1708259329.468253,WE7TBc2CcPNPnw,None
5,3,Person_3,33,City_B,1708259330.0940356,j381sFn7OXpq4Q,Job_3
6,4,Person_4,34,City_B,1708259330.0940356,4etCha3RnGfliQ,Job_4
7,5,Person_5,35,City_B,1708259330.0940356,sJgeWj//yKrp7w,Job_5
8,6,Person_6,36,City_B,1708259330.0940356,la0x2vsL2NxpkA,Job_6
9,7,Person_7,37,City_B,1708259330.0940356,gAiPHJ9AkegN7A,Job_7


# Solution: First make sure that the following modules are installed:

In [9]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [ ]:
# to do: homework :)

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX